In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, ShuffleSplit
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import make_scorer
from sklearn.neural_network import MLPRegressor

In [2]:
path_eval = "/kaggle/input/dataprojet/evaluation.csv"
path_clean = "/kaggle/input/df-no-out-norms/development_noNoise_noRMS_noOutliers.csv"

In [3]:
def prediction(eval_path, results_path, model):
    df_test = pd.read_csv(eval_path)
    ## Remove noise columns
    ix = 0,7,12,15,16,17
    noise = [f"rms[{i}]" for i in range(18)]
    noise += [f"pmax[{i}]" for i in ix]
    noise += [f"negpmax[{i}]" for i in ix]
    noise += [f"tmax[{i}]" for i in ix]
    noise += [f"area[{i}]" for i in ix]
    df_test = df_test.drop(columns = noise)
    y_pred_eval = model.predict(df_test.drop(columns = ["Id"]))
    df_test[["x","y"]] =  y_pred_eval
    df_test['Predicted'] = df_test['x'].astype(str) + '|' + df_test['y'].astype(str)
    df_test[['Id', 'Predicted']].to_csv(results_path, index = False)

In [4]:
## Metric to be used to evaluate the performances of models 
def distance(y_true, y_pred):
    n = len(y_true)
    d = np.sum(np.sqrt(np.sum((y_true - y_pred)**2, axis=1))) / n
    return d
scorer = make_scorer(distance)

In [5]:
from sklearn.linear_model import Ridge
def multi_pipeline_train(df_path, model, tuning_params=None, train_size=0.7):
    # Load the data
    df = pd.read_csv(df_path)
    np.random.seed(124)
    print("Start performing a hypeparameter Tunning for multi output regresor using RF")
    ## Split the dataset
    X = df.drop(columns=["x", "y"])
    y = df[["x", "y"]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, random_state=124, shuffle = True)

    print("Data splited")
    # Multi-output regressor without hyperparameter tuning
    if tuning_params is None:
        print("Trainig the base model")
        multi_output_reg = MultiOutputRegressor(model)
        multi_output_reg.fit(X_train, y_train)
        y_pred = multi_output_reg.predict(X_test)
        distance_metric = distance(y_test, y_pred)
        print(f"Base Multi-output regressor using RF distance on test dataset: {distance_metric}")
        return multi_output_reg

    # Multi-output regressor with hyperparameter tuning
    else:
        print("Hyperparamter Tunning Performing start")
        multi_output_reg = MultiOutputRegressor(model)
        cvparam = ShuffleSplit(test_size=0.3, n_splits=1, random_state=124)
        grid_search = GridSearchCV(estimator=multi_output_reg, param_grid=tuning_params, cv=cvparam,
                                   scoring=scorer, n_jobs=-1, verbose = 4)
        grid_search.fit(X, y)
        best_model = grid_search.best_estimator_
        #distance_metric_tuned = distance(y_test, y_pred_best)
       # print(f"Tuned Multi-output regressor using RF distance on test dataset: {distance_metric_tuned}")
        print("Best Hyperparameters found:", grid_search.best_params_)
        return best_model, grid_search.best_params_

In [6]:
rf_regressor = RandomForestRegressor(random_state = 42)
model_base = multi_pipeline_train(path_clean, rf_regressor)

Start performing a hypeparameter Tunning for multi output regresor using RF


Data splited
Trainig the base model


Base Multi-output regressor using RF distance on test dataset: 4.689793757257982


In [7]:
result_path = "/kaggle/working/rf_base_eval.csv"
prediction(path_eval, result_path, model_base)
print("Base model evaluation saved on", result_path)

Base model evaluation saved on /kaggle/working/rf_base_eval.csv


In [8]:
rf_param_grid = {
    'estimator__n_estimators': [200, 300, 500],
    'estimator__max_depth': [None, 10, 20],
}

best_model_rf, hyperparameters_rf = multi_pipeline_train(path_clean, rf_regressor, tuning_params=rf_param_grid)

Start performing a hypeparameter Tunning for multi output regresor using RF


Data splited
Hyperparamter Tunning Performing start
Fitting 1 folds for each of 9 candidates, totalling 9 fits


[CV 1/1] END estimator__max_depth=None, estimator__n_estimators=300;, score=4.658 total time=244.8min
[CV 1/1] END estimator__max_depth=20, estimator__n_estimators=200;, score=4.718 total time=163.8min


/opt/conda/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV 1/1] END estimator__max_depth=None, estimator__n_estimators=500;, score=4.651 total time=410.5min


[CV 1/1] END estimator__max_depth=None, estimator__n_estimators=200;, score=4.667 total time=164.9min
[CV 1/1] END estimator__max_depth=10, estimator__n_estimators=500;, score=8.673 total time=267.8min


[CV 1/1] END estimator__max_depth=10, estimator__n_estimators=200;, score=8.680 total time=109.5min
[CV 1/1] END estimator__max_depth=10, estimator__n_estimators=300;, score=8.677 total time=161.6min
[CV 1/1] END estimator__max_depth=20, estimator__n_estimators=300;, score=4.710 total time=219.5min


In [ ]:
result_path = "/kaggle/working/rf_best_eval.csv"
prediction(path_eval, result_path, best_model_rf)
print("Base model evaluation saved on", result_path)